In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test= pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
test.shape

(418, 11)

In [8]:
train=train.drop(['Cabin', 'Ticket'], axis=1)
test=test.drop(['Cabin', 'Ticket'], axis=1)

In [9]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S


In [10]:
train['Title']=train['Name'].str.extract(r'(\w+)\.')
test['Title']=test['Name'].str.extract(r'(\w+)\.')

In [11]:
train['Title'].value_counts()

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: count, dtype: int64

In [12]:
train['Title']=train['Title'].replace([ 'Lady','Don', 'Rev', 'Dr', 'Major','Col', 'Sir','Capt', 'Countess','Jonkheer'], 'rare')
train['Title']=train['Title'].replace(['Mlle','Ms'], 'Mrs')
train['Title']=train['Title'].replace('Mme', 'Miss')
test['Title']=test['Title'].replace([ 'Lady','Don','Dona', 'Rev', 'Dr', 'Major','Col', 'Sir','Capt', 'Countess','Jonkheer'], 'rare')
test['Title']=test['Title'].replace(['Mlle','Ms'], 'Mrs')
test['Title']=test['Title'].replace('Mme', 'Miss')

In [13]:
test['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'rare'], dtype=object)

In [14]:
train['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'rare'], dtype=object)

In [15]:
train=train.drop('Name', axis=1)
test=test.drop('Name', axis=1)

In [16]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,male,22.0,1,0,7.2500,S,Mr
1,2,1,1,female,38.0,1,0,71.2833,C,Mrs
2,3,1,3,female,26.0,0,0,7.9250,S,Miss
3,4,1,1,female,35.0,1,0,53.1000,S,Mrs
4,5,0,3,male,35.0,0,0,8.0500,S,Mr


In [17]:
from sklearn.preprocessing import LabelEncoder

### Label Encoder

In [18]:
Enco=LabelEncoder()

In [19]:
Enco.fit(train['Sex'])
train['Sex']=Enco.transform(train['Sex'])
test['Sex']=Enco.transform(test['Sex'])

In [20]:
Enco.fit(train['Embarked'])
train['Embarked']=Enco.transform(train['Embarked'])
test['Embarked']=Enco.transform(test['Embarked'])

In [21]:
Enco.fit(train['Title'])
train['Title']=Enco.transform(train['Title'])
test['Title']=Enco.transform(test['Title'])

In [22]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,1,22.0,1,0,7.2500,2,2
1,2,1,1,0,38.0,1,0,71.2833,0,3
2,3,1,3,0,26.0,0,0,7.9250,2,1
3,4,1,1,0,35.0,1,0,53.1000,2,3
4,5,0,3,1,35.0,0,0,8.0500,2,2


In [23]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,892,3,1,34.5,0,0,7.8292,1,2
1,893,3,0,47.0,1,0,7.0000,2,3
2,894,2,1,62.0,0,0,9.6875,1,2
3,895,3,1,27.0,0,0,8.6625,2,2
4,896,3,0,22.0,1,1,12.2875,2,3


In [24]:
from sklearn.impute import KNNImputer

In [25]:
Imp=KNNImputer(n_neighbors=5)
train[ ['Pclass', 'Sex', 'Age', 'Title']]=Imp.fit_transform(train[ ['Pclass', 'Sex', 'Age', 'Title']])
test[ ['Pclass', 'Sex', 'Age', 'Title']]=Imp.fit_transform(test[ ['Pclass', 'Sex', 'Age', 'Title']])

In [26]:
from sklearn.impute import SimpleImputer
SimImp=SimpleImputer()
test[['Fare']]=SimImp.fit_transform(test[['Fare']])

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    float64
 3   Sex          891 non-null    float64
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    int64  
 9   Title        891 non-null    float64
dtypes: float64(5), int64(5)
memory usage: 69.7 KB


In [28]:
train.loc[[0]]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3.0,1.0,22.0,1,0,7.25,2,2.0


In [29]:
from sklearn.svm import LinearSVC

In [30]:
SVM=LinearSVC(tol=1e-7)
SVM.fit(X_train, y_train)
y_pred_svm=SVM.predict(X_train)

NameError: name 'X_train' is not defined

In [ ]:
metrics.accuracy_score(y_train, y_pred_svm)